In [1]:
import os
import cv2
from subprocess import call
from ultralytics import YOLO
import tensorflow.compat.v1 as tf

In [24]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# Example of a simple TensorFlow 1.x model
x = tf.placeholder(tf.float32, shape=[None, 10], name='x')
y = tf.placeholder(tf.float32, shape=[None, 1], name='y')
weights = tf.Variable(tf.random_normal([10, 1]), name='weights')
bias = tf.Variable(tf.zeros([1]), name='bias')

predictions = tf.add(tf.matmul(x, weights), bias, name='predictions')
loss = tf.reduce_mean(tf.square(predictions - y), name='loss')
train_op = tf.train.AdamOptimizer(0.001).minimize(loss)


In [25]:
# Save model
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, 'saved_model/my_model.ckpt')


In [26]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, 'saved_model/my_model.ckpt')
    print("Model restored successfully.")


INFO:tensorflow:Restoring parameters from saved_model/my_model.ckpt
Model restored successfully.


In [27]:
class SteeringAnglePredictor:
    def __init__(self, model_path):
        """
        Initialize the predictor by restoring a TensorFlow 1.x model.

        Args:
            model_path (str): Path to the saved TensorFlow 1.x model checkpoint.
        """
        self.session = tf.Session()  # Create a session for TensorFlow 1.x

        # Restore the graph and variables
        saver = tf.train.import_meta_graph(model_path + '.meta')
        saver.restore(self.session, model_path)

        # Access tensors from the restored graph
        self.graph = tf.get_default_graph()
        self.x = self.graph.get_tensor_by_name('x:0')
        self.keep_prob = self.graph.get_tensor_by_name('keep_prob:0')  # Replace as needed
        self.y = self.graph.get_tensor_by_name('y:0')

        self.smoothed_angle = 0

    def predict_angle(self, image):
        """
        Predict the steering angle from a preprocessed image.

        Args:
            image (numpy.ndarray): Preprocessed input image.

        Returns:
            float: Predicted steering angle in degrees.
        """
        predicted_radians = self.session.run(self.y, feed_dict={self.x: [image], self.keep_prob: 1.0})[0][0]
        return predicted_radians * 180.0 / 3.14159265  # Convert radians to degrees

    def smooth_angle(self, predicted_angle):
        """
        Smooth the predicted steering angle for consistent control.

        Args:
            predicted_angle (float): Raw predicted angle in degrees.

        Returns:
            float: Smoothed steering angle.
        """
        if self.smoothed_angle == 0:
            self.smoothed_angle = predicted_angle
        else:
            self.smoothed_angle += 0.2 * pow(abs(predicted_angle - self.smoothed_angle), 2.0 / 3.0) * (
                (predicted_angle - self.smoothed_angle) / abs(predicted_angle - self.smoothed_angle)
            )
        return self.smoothed_angle

    def close(self):
        """Close the TensorFlow session."""
        self.session.close()


In [18]:
class DrivingSimulator:
    def __init__(self, predictor, data_dir, steering_image_path, is_windows=False):
        self.predictor = predictor
        self.data_dir = data_dir
        self.steering_image = cv2.imread(steering_image_path, 0)
        self.is_windows = is_windows
        self.rows, self.cols = self.steering_image.shape

    def start_simulation(self):
        i = 0
        while cv2.waitKey(10) != ord('q'):
            full_image = cv2.imread(os.path.join(self.data_dir, f"{i}.jpg"))
            resized_image = cv2.resize(full_image[-150:], (200, 66)) / 255.0

            predicted_angle = self.predictor.predict_angle(resized_image)
            smoothed_angle = self.predictor.smooth_angle(predicted_angle)

            if not self.is_windows:
                call("clear")
            print(f"Predicted steering angle: {predicted_angle} degrees")

            self.display_frames(full_image, smoothed_angle)
            i += 1

        cv2.destroyAllWindows()
        
        def display_frames(self, full_image, smoothed_angle):
            cv2.imshow("frame", full_image)
            rotation_matrix = cv2.getRotationMatrix2D((self.cols / 2, self.rows / 2), -smoothed_angle, 1)
            rotated_steering_wheel = cv2.warpAffine(self.steering_image, rotation_matrix, (self.cols, self.rows))
            cv2.imshow("steering wheel", rotated_steering_wheel)

In [30]:
if __name__ == "__main__":
    tf.disable_v2_behavior()
    model_path = 'saved_model/regression_model/model_22.keras'
    data_dir = "data/driving_dataset"
    steering_image_path = "data/steering_wheel_image.jpg"

    # Determine if running on Windows
    is_windows = os.name == 'nt'

    predictor = SteeringAnglePredictor(model_path)
    simulator = DrivingSimulator(predictor, data_dir, steering_image_path, is_windows)

    try:
        simulator.start_simulation()
    finally:
        predictor.close()

OSError: File does not exist. Received: saved_model/regression_model/model_22.keras.meta.

In [31]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Rebuild the graph manually
x = tf.placeholder(tf.float32, shape=[None, 10], name='x')
y = tf.placeholder(tf.float32, shape=[None, 1], name='y')
weights = tf.Variable(tf.random_normal([10, 1]), name='weights')
bias = tf.Variable(tf.zeros([1]), name='bias')
predictions = tf.add(tf.matmul(x, weights), bias, name='predictions')

# Restore the model variables
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, 'saved_model/regression_model/model_22.keras')
    print("Model restored successfully!")


INFO:tensorflow:Restoring parameters from saved_model/regression_model/model_22.keras


DataLossError: Graph execution error:

Detected at node 'save_2/RestoreV2' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    File "C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    File "C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    File "C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    File "C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever
    File "C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922

In [32]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file(file_name='saved_model/regression_model/model_22.keras', tensor_name='', all_tensors=True)

Unable to open table file saved_model\regression_model\model_22.keras: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
